In [1]:
import sqlite3
conn = sqlite3.connect('airline2.db')

In [2]:
import pandas as pd

airports = pd.read_csv("airports.csv")
carriers = pd.read_csv("carriers.csv")
airplanes = pd.read_csv("planedata.csv")
df1 = pd.read_csv("2006.csv")
df2 = pd.read_csv("2007.csv")
df3 = pd.read_csv("2008.csv")



In [3]:
ontime = pd.concat([df1, df2, df3])

In [4]:
ontime.tail

<bound method NDFrame.tail of          Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
0        2006      1          11          3    743.0         745   1024.0   
1        2006      1          11          3   1053.0        1053   1313.0   
2        2006      1          11          3   1915.0        1915   2110.0   
3        2006      1          11          3   1753.0        1755   1925.0   
4        2006      1          11          3    824.0         832   1015.0   
...       ...    ...         ...        ...      ...         ...      ...   
2389212  2008      4          17          4   1025.0        1025   1234.0   
2389213  2008      4          17          4   1319.0        1320   1527.0   
2389214  2008      4          17          4   1335.0        1335   1556.0   
2389215  2008      4          17          4   1933.0        1935   2140.0   
2389216  2008      4          17          4    621.0         615    752.0   

         CRSArrTime UniqueCarrier  FlightNum 

In [5]:
airports.to_sql('airports', con = conn, index = False) 
carriers.to_sql('carriers', con = conn, index = False)
airplanes.to_sql('planedata', con = conn, index = False)
ontime.to_sql('ontime', con = conn, index = False) 

In [6]:
conn.commit()

In [7]:
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

[('airports',), ('carriers',), ('planedata',), ('ontime',)]

In [8]:
# City with highest number of inbound flights (excluding cancelled flights) = Chicago

q1 = c.execute('''
SELECT airports.city as city, COUNT(*) AS total
FROM airports  JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC
''').fetchall()

pd.DataFrame(q1)

,0,1
0,Chicago,1051340
1,Atlanta,937377
2,Dallas-Fort Worth,671820
3,Houston,609759
4,New York,547471
...,...,...
296,New Haven,3
297,Greenville,2
298,Scottsbluff,1
299,North Platte,1


In [9]:
# airplane that has the lowest avg. dep. delay (excl. cancelled and diverted) = 737-230

q3 = c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planedata JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
 ''').fetchall()

pd.DataFrame(q3)

,0,1
0,737-230,12.956403
1,737-282,13.688697
2,737-236,13.950983
3,737-2P6,15.585598
4,737-2X6C,15.667864
...,...,...
161,ATR-72-212,47.663491
162,EMB-135ER,47.791864
163,DHC-8-102,52.545455
164,A330-323,54.285714


In [10]:
# Company with the highest number of cancelled flights = American Airlines

q4 = c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1 AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
ORDER BY total DESC 
 ''').fetchall()

pd.DataFrame(q4)

,0,1
0,American Airlines Inc.,38835
1,United Air Lines Inc.,26783
2,Delta Air Lines Inc.,17267
3,Pinnacle Airlines Inc.,12036


### q2 below is assumed to be q4 mentioned in the assignment.
__
##### It's also the answer to question 4 of the assignment (simplified version of the code)

In [42]:
# This is basicly just q4 (line 10) / with q5 (the code below):

q5 = c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
ORDER BY total DESC 
 ''').fetchall()
z4 = pd.DataFrame(q4)
z5 = pd.DataFrame(q5)
q2 = z4[1]/z5[1]

q2


0    0.026205
1    0.023411
2    0.015235
3    0.034665
Name: 1, dtype: float64

In [43]:
# The answer is no. 3 = Pinnacle Airlines 